<a href="https://colab.research.google.com/github/githbsingh/bottleneckSolution/blob/main/sharding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# SHARDING based on Region
import sqlite3

# Sample data
users = [
    (1, 'Alice', 'US'),
    (2, 'Bob', 'EU'),
    (3, 'Charlie', 'US'),
    (4, 'Diana', 'ASIA'),
    (5, 'Ethan', 'EU')
]

# Dropping Tables to avoid redundancy from each regional databases
conn = sqlite3.connect("users_asia.db")
cursor = conn.cursor()
cursor.execute('''
    DROP TABLE Users''')

# Dropping Tables to avoid redundancy from each regional databases
conn = sqlite3.connect("users_us.db")
cursor = conn.cursor()
cursor.execute('''
    DROP TABLE Users''')

# Dropping Tables to avoid redundancy from each regional databases
conn = sqlite3.connect("users_eu.db")
cursor = conn.cursor()
cursor.execute('''
    DROP TABLE Users''')


# Create a function to insert into the right shard
def insert_user(user_id, name, region):
    db_name = f"users_{region.lower()}.db"
    #print(db_name)
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Create table if not exists

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Users (
            user_id INTEGER,
            name TEXT,
            region TEXT
        )
    ''')

    # Insert user
    cursor.execute('INSERT INTO Users (user_id, name, region) VALUES (?, ?, ?)', (user_id, name, region))
    conn.commit()
    conn.close()
    print(f"User {name} inserted into {db_name}")

# Insert all users into their respective shards
for user in users:
    insert_user(*user)

print("Connecting to database in US")
conn = sqlite3.connect('users_us.db')
cursor = conn.cursor()
cursor.execute('SELECT * FROM Users')
print(cursor.fetchall())
conn.close()

print("Connecting to database in EU")
conn = sqlite3.connect('users_eu.db')
cursor = conn.cursor()
cursor.execute('SELECT * FROM Users')
print(cursor.fetchall())
conn.close()



User Alice inserted into users_us.db
User Bob inserted into users_eu.db
User Charlie inserted into users_us.db
User Diana inserted into users_asia.db
User Ethan inserted into users_eu.db
Connecting to database in US
[(1, 'Alice', 'US'), (3, 'Charlie', 'US')]
Connecting to database in EU
[(2, 'Bob', 'EU'), (5, 'Ethan', 'EU')]
